Сделано на основе ноутбука от unsloth

https://unsloth.ai/blog/r1-reasoning

Сделано на основе ноутбука от unsloth

https://unsloth.ai/blog/r1-reasoning

Установим все необходимые библиотеки  
unsloth - Для оптимизации тренировки  
vllm - для инференса модели  
tensorboard - для логирования и визуализации  
trl - библиотека для тренировки LLM 

In [1]:
# !pip install unsloth vllm tensorboard
# !pip install --upgrade pillow

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)
# Импортируем наши генераторы конкретных типов задач:
from re_rl.tasks.generators import (
    generate_random_linear_task,
    generate_random_futoshiki_task,
    generate_random_knights_knaves_task,
    # ... при желании остальные ...
)
# Импортируем глобальную функцию compute_reward_for_task (или аналог),
# которая умеет проверять ответ для каждого task_type:
from re_rl.rewards import compute_reward_for_task


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-25 21:32:12 [__init__.py:256] Automatically detected platform cuda.
WARNING 03-25 21:32:12 [cuda.py:394] Detected different devices in the system: NVIDIA GeForce RTX 3090 Ti, NVIDIA GeForce RTX 3090. Please make sure to set `CUDA_DEVICE_ORDER=PCI_BUS_ID` to avoid unexpected behavior.


Мы хотим попробовать потренировать GRPO дома на 3090/4090 с 24ГБ видеопамяти.
Будем тренировать не всю модель, а LoRA адаптер. В таком режиме веса модели замораживаются, а тренируются дополнительные матрицы, которые затем будут добавлены в целевые веса модели.

С последними обновлениями unsloth для GRPO стало возможным использовать модели прямо с очень большим контекстом. В 3090 влезала 3B модель с 15000 контекстом.

Варьируйте параметры max_seq_len, gpu_memory_utilization если параметры установленные по-умолчанию в память не влезают.
А вообще - варьируйте все и ресечьте)

1.5B моделька с общим контекстом 456 будет трениться на всем сете GSM8K-ru примерно ~ часов.
+ если включать промежуточный евал на тестсете один прогон занимает минут 40-50.

In [3]:
from unsloth import is_bfloat16_supported
import torch

max_seq_length = 2000 # параметр задает длину контекста модели. Чем больше тем больше памяти будет требоваться и медленне тренироваться
lora_rank = 64 # LoRA ранг 64 - довольно большой, у нас получится ~120 миллионов тренируемых параметров.

# model_name = "Qwen/Qwen2.5-7B-Instruct" # Большая 7B модель
# model_name = "Qwen/Qwen2.5-3B-Instruct" # 3B модель
model_name = "Qwen/Qwen2.5-1.5B-Instruct" # 1.5B модель
# model_name = "Qwen/Qwen2.5-0.5B-Instruct" # 0.5B модель, самая слабая, но быстрее всего учится

model_name = "/media/user/My Passport2/hfmodels/Qwen2.5-1.5B-Instruct/"
# один из важнейших параметров далее - gpu_memory_utilization.
# расчеты из того что у нас доступно 24ГБ видеопамяти. Если меньше или больше - варьируйте значение.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # Загружаем модель в 4-бит режиме
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # сколько памяти будет занимать модель на видеокарте, можно варьировать
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # список модулей к которым применяется LoRA
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.18: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 3090 Ti. Num GPUs = 2. Max memory: 23.677 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /media/user/My Passport2/hfmodels/Qwen2.5-1.5B-Instruct/ with actual GPU utilization = 43.24%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.68 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2000. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.22 GB. Also swap space = 5 GB.
INFO 03-25 21:32:20 [config.py:583] This model supports multiple tasks: {'classify', 'embed', 'reward', 'score', 'generate'}. Defaulting to 'generate'.
WARNING 03

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-25 21:32:21 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-25 21:32:21 [model_runner.py:1146] Model loading took 1.2130 GB and 0.448007 seconds
INFO 03-25 21:32:22 [worker.py:267] Memory profiling takes 0.98 seconds
INFO 03-25 21:32:22 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.43) = 10.24GiB
INFO 03-25 21:32:22 [worker.py:267] model weights take 1.21GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 7.93GiB.
INFO 03-25 21:32:23 [executor_base.py:111] # cuda blocks: 18554, # CPU blocks: 11702
INFO 03-25 21:32:23 [executor_base.py:116] Maximum concurrency for 2000 tokens per request: 148.43x
INFO 03-25 21:32:24 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. I

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:14<00:00,  1.86it/s]

INFO 03-25 21:32:38 [model_runner.py:1570] Graph capturing finished in 14 secs, took 2.83 GiB
INFO 03-25 21:32:38 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 17.07 seconds



Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
###################################################
# Шаг 3: Генерация задач
###################################################

# Допустим, мы хотим потренироваться на ЛИНЕЙНЫХ уравнениях (как пример).
# Напишем функцию, которая сделает N тренировочных и M валидационных:
import random
from typing import List, Dict
from re_rl.rewards import (
    reward_format_check,
    reward_cot_quality,
    reward_correctness,
)



def make_linear_dataset(num_train=200, num_eval=50, language="ru", detail_level=3):
    train_items = []
    for _ in range(num_train):
        t = generate_random_linear_task(language=language, detail_level=detail_level)
        r = t.get_result()
        # Сохраняем нужные поля
        train_items.append({
            "task_type": t.get_task_type(),       # "linear"
            "problem": r["problem"],             # описание задачи
            "prompt": r["prompt"],               # "Задача: ..."
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    eval_items = []
    for _ in range(num_eval):
        t = generate_random_linear_task(language=language, detail_level=detail_level)
        r = t.get_result()
        eval_items.append({
            "task_type": t.get_task_type(),
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    return train_items, eval_items

# Аналогично можете сделать make_futoshiki_dataset, make_knights_knaves_dataset, ...
# Или же все объединить в одну функцию с параметром.


###################################################
# Шаг 4: Преобразование в формат unsloth
###################################################
SYSTEM_PROMPT = """Отвечай строго в формате:
<reasoning>
(Шаги решения)
</reasoning>
<answer>
(Короткий итоговый ответ)
</answer>
"""


def convert_to_unsloth_format(dataset_list: List[Dict]):
    """
    На вход: list, где каждый dict имеет 
      {
       "task_type": str, 
       "problem": str,
       "prompt": str,
       "final_answer": str,
       ...
      }
    Возвращаем список, пригодный для GRPOTrainer: 
      [
        {
          "prompt": [
             {"role": "system", "content": SYSTEM_PROMPT},
             {"role": "user",   "content": item["prompt"],
               "metadata": {
                  "task_type": item["task_type"],
                  "problem": item["problem"],
                  "ref_final_answer": item["final_answer"]
                }
             }
          ],
          "answer": item["final_answer"]
        },
        ...
      ]
    """
    out = []
    for d in dataset_list:
        user_msg = {
            "role": "user",
            "content": d["prompt"],
            "metadata": {
                "task_type": d["task_type"],
                "problem": d["problem"],
                "ref_final_answer": d["final_answer"]
            }
        }
        system_msg = {
            "role": "system",
            "content": SYSTEM_PROMPT
        }
        ex = {
            "prompt": [system_msg, user_msg],
            "answer": d["final_answer"]  # "gold"
        }
        out.append(ex)
    return out

###################################################
# Шаг 5: список reward-функций, вызывающая
###################################################
reward_funcs_list = [
    reward_format_check,   # проверка, что есть <reasoning>...</reasoning> и <answer>...</answer>
    reward_cot_quality,    # небольшой бонус за содержательный reasoning
    reward_correctness,    # основная награда за правильное решение
]


###################################################
# Шаг 6: Собственно обучение GRPO на линейных задачах
###################################################
# 1) генерируем датасет
train_lin, eval_lin = make_linear_dataset(num_train=1000, num_eval=50, language="ru", detail_level=3)
# 2) конвертируем
train_lin_unsloth = convert_to_unsloth_format(train_lin)
eval_lin_unsloth  = convert_to_unsloth_format(eval_lin)

In [5]:
print(len(train_lin_unsloth))

1000


In [6]:
from trl import GRPOConfig, GRPOTrainer
# 3) Config
train_args_lin = GRPOConfig(
    use_vllm = True,
    vllm_gpu_memory_utilization = 0.3,
    learning_rate = 2e-5,
    num_train_epochs = 1,
    logging_steps = 10,
    save_steps = 50,
    bf16 = True,    # если GPU поддерживает bf16
    fp16 = False,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_generations = 8,
    max_prompt_length = 1000,
    max_completion_length = 1000,
    output_dir = "outputs_linear_only",
)

trainer_lin = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = reward_funcs_list,  # список из одной функции
    args = train_args_lin,
    train_dataset = train_lin_unsloth,
    eval_dataset  = eval_lin_unsloth,
)
trainer_lin.train(resume_from_checkpoint=False)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 73,859,072/5,000,000,000 (1.48% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_format_check,rewards / reward_cot_quality,rewards / reward_correctness
10,0.000000,0.795781,0.468305,162.153125,0.000740,0.169063,0.085469,0.541250
20,0.000200,0.937500,0.341033,154.942187,0.004576,0.198750,0.099688,0.639062
30,0.000500,1.095625,0.429912,147.239062,0.013708,0.198125,0.099688,0.797812
40,0.000900,1.253750,0.432612,140.735938,0.023145,0.196563,0.099063,0.958125
50,0.001700,1.375313,0.326483,132.918750,0.042795,0.197813,0.100000,1.077500
60,0.001500,1.470469,0.265438,125.031250,0.038632,0.199063,0.099844,1.171562
70,0.001800,1.478438,0.249083,138.023438,0.044940,0.199375,0.100000,1.179062
80,0.001700,1.483750,0.242816,140.032813,0.041594,0.199688,0.100000,1.184062
90,0.001600,1.530625,0.157024,134.068750,0.039088,0.199688,0.100000,1.230937
100,0.001700,1.511563,0.184734,136.468750,0.043080,0.199688,0.099844,1.212031


TrainOutput(global_step=125, training_loss=0.001271246774122119, metrics={'train_runtime': 2543.5643, 'train_samples_per_second': 0.393, 'train_steps_per_second': 0.049, 'total_flos': 0.0, 'train_loss': 0.001271246774122119})

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Сколько раз р встречается в слове стравберри?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  8.50it/s, est. speed input: 349.83 toks/s, output: 153.57 toks/s]


'В слове "стравберри" р встречается 2 раза.'

In [8]:
model.save_lora("grpo_saved_lora")

In [11]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Сколько раз р встречается в слове стравберри?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it, est. speed input: 68.68 toks/s, output: 110.48 toks/s]


'<reasoning>\n1. Начнем с анализа слова "стравберри".\n2. В слове "стравберри" мы можем увидеть букву "р" в следующих местах:\n   - В начале слова "стравберри"\n   - В конце слова "стравберри"\n3. Таким образом, буква "р" встречается в слове "стравберри" 2 раза.\n</reasoning>\n<answer>\n2\n</answer>'